In [9]:
import pytesseract
import cv2
from PIL import Image
import os
import subprocess
import time
import csv
from datetime import datetime
import re

In [10]:
# def select_roi(image):
#     roi = cv2.selectROI("Select ROI", image, showCrosshair=True)
#     cv2.destroyWindow("Select ROI")
#     return roi

In [11]:
def select_roi(image):
    # 원본 이미지의 크기 가져오기
    height, width = image.shape[:2]
    
    # 목표 크기 설정 (예: 1280x720)
    target_width = 1280
    target_height = 720
    
    # 이미지가 목표 크기보다 큰 경우 크기 조절
    if width > target_width or height > target_height:
        # 비율 유지하면서 크기 조절
        scale = min(target_width / width, target_height / height)
        new_width = int(width * scale)
        new_height = int(height * scale)
        
        # 이미지 리사이즈
        display_image = cv2.resize(image, (new_width, new_height))
    else:
        display_image = image.copy()
        scale = 1.0

    # ROI 선택
    roi = cv2.selectROI("Select ROI", display_image, showCrosshair=True)
    cv2.destroyWindow("Select ROI")
    
    # 원본 이미지에 대한 ROI 좌표로 변환
    if scale != 1.0:
        roi = tuple(int(x / scale) for x in roi)
    
    return roi

In [ ]:
def capture_screenshot(adb_path):
    try:
        subprocess.run([adb_path, 'exec-out', 'screencap', '-p'], stdout=open('screen.png', 'wb'))
        return True
    except Exception as e:
        print(f"스크린샷 캡처 에러: {e}")
        return False

In [ ]:
def process_image(image_path, regions):
    if not os.path.exists(image_path):
        print("오류: screen.png 파일이 존재하지 않습니다.")
        return None

    img = cv2.imread(image_path)
    if img is None:
        print("오류: 이미지를 로드할 수 없습니다.")
        return None

    results = {}
    for key, (x, y, w, h) in regions.items():
        roi = img[y:y+h, x:x+w]
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        binary = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
        )
        
        pil_image = Image.fromarray(gray)
        text = pytesseract.image_to_string(pil_image, lang='kor+eng')
        
        # pil_image = Image.fromarray(binary)
        # text = pytesseract.image_to_string(pil_image, lang='kor+eng', config='--psm 6')
        results[key] = text.strip()

    try:
        battery_power = ''
        if '내장 배터리용 전원 충전' in results:
            tmp = results['내장 배터리용 전원 충전'].replace(',', '')
            # 'W'를 제거하고 숫자만 추출
            for word in tmp.split():
                # 'W'가 포함된 경우 처리
                if 'W' in word:
                    battery_power = word.replace('W', '')  # 'W' 제거
                    break  # 첫 번째로 찾은 값만 사용
            if battery_power:  # battery_power가 비어있지 않은 경우
                results["생산전압"] = format(float(battery_power) / 5, '.13f')  # 소수점 이하 13자리로 포맷
            else:
                results["생산전압"] = "계산 불가"  # battery_power가 비어있을 경우
        else:
            results["생산전압"] = "계산 불가"  # 키가 없을 경우
    except (ValueError, KeyError):
        results["생산전압"] = "계산 불가"
    
    return results